In [ ]:
import yfinance as yf
import pandas as pd

data = yf.download("^NSEI", period="10y", interval="1d")

#downloading more global indices data 
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta


end_date = datetime.today().strftime('2025-10-1')
start_date = (datetime.today() - timedelta(days=365*10)).strftime('2015-01-1')


# Download additional market data
def fetch_market_context_data(start_date, end_date):

    # Indian indices 
    bank_nifty = yf.download("^NSEBANK", start=start_date, end=end_date)['Close']
    india_vix = yf.download("^INDIAVIX", start=start_date, end=end_date)['Close']
    
    # Global indices (for context)
    sp500 = yf.download("^GSPC", start=start_date, end=end_date)['Close']
    nasdaq = yf.download("^IXIC", start=start_date, end=end_date)['Close']
    
    # Commodities & Currency
    usdinr = yf.download("INR=X", start=start_date, end=end_date)['Close']
    crude_oil = yf.download("CL=F", start=start_date, end=end_date)['Close']
    gold = yf.download("GC=F", start=start_date, end=end_date)['Close']
    
    return {
        'bank_nifty': bank_nifty,
        'india_vix': india_vix,
        'sp500': sp500,
        'nasdaq': nasdaq,
        'usdinr': usdinr,
        'crude_oil': crude_oil,
        'gold': gold
    }

# Fetch market data dynamically (10 years)
market_data = fetch_market_context_data('2015-01-01', '2025-01-01')


/tmp/ipykernel_29461/2639922344.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download("^NSEI", period="10y", interval="1d")
[*********************100%***********************]  1 of 1 completed
/tmp/ipykernel_29461/2639922344.py:21: FutureWarning: YF.download() has changed argument auto_adjust default to True
  bank_nifty = yf.download("^NSEBANK", start=start_date, end=end_date)['Close']
[*********************100%***********************]  1 of 1 completed
/tmp/ipykernel_29461/2639922344.py:22: FutureWarning: YF.download() has changed argument auto_adjust default to True
  india_vix = yf.download("^INDIAVIX", start=start_date, end=end_date)['Close']
[*********************100%***********************]  1 of 1 completed
/tmp/ipykernel_29461/2639922344.py:25: FutureWarning: YF.download() has changed argument auto_adjust default to True
  sp500 = yf.download("^GSPC", start=start_date, end=end_date)['Close']
[*********************100%*********

In [ ]:
#Data cleaning
data = data.ffill().bfill()
data =data.dropna() 
# vol cleaning
data['Volume'] = data['Volume'].ffill().bfill()
print(data.head())




Price             Close         High          Low         Open  Volume
Ticker            ^NSEI        ^NSEI        ^NSEI        ^NSEI   ^NSEI
Date                                                                  
2015-10-21  8251.700195  8294.400391  8217.150391  8258.349609  144800
2015-10-23  8295.450195  8328.099609  8280.750000  8308.250000  152000
2015-10-26  8260.549805  8336.299805  8252.049805  8333.650391  133900
2015-10-27  8232.900391  8241.950195  8217.049805  8230.349609  156700
2015-10-28  8171.200195  8209.099609  8131.799805  8188.899902  188900


In [ ]:
#validate and correct irregularities
# Check rows where Volume is zero
print(data[data['Volume'] == 0])
data['Volume'].replace(0, pd.NA, inplace=True)
data['Volume'] = data['Volume'].ffill()

#  using z-score to detect outlier
from scipy import stats
import numpy as np
z_scores = np.abs(stats.zscore(data[['Open','High','Low','Close','Volume']]))
print(np.where(z_scores > 3))

duplicates = data.index.duplicated()
print(data[duplicates])
#removed duplicate data
data = data[~duplicates]


/tmp/ipykernel_29461/1664061200.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Volume'].replace(0, pd.NA, inplace=True)


Price      Close  High   Low  Open Volume
Ticker     ^NSEI ^NSEI ^NSEI ^NSEI  ^NSEI
Date                                     
2015-10-21   NaN   NaN   NaN   NaN    NaN
2015-10-23   NaN   NaN   NaN   NaN    NaN
2015-10-26   NaN   NaN   NaN   NaN    NaN
2015-10-27   NaN   NaN   NaN   NaN    NaN
2015-10-28   NaN   NaN   NaN   NaN    NaN
...          ...   ...   ...   ...    ...
2025-10-15   NaN   NaN   NaN   NaN    NaN
2025-10-16   NaN   NaN   NaN   NaN    NaN
2025-10-17   NaN   NaN   NaN   NaN    NaN
2025-10-20   NaN   NaN   NaN   NaN    NaN
2025-10-21   NaN   NaN   NaN   NaN    NaN

[2463 rows x 5 columns]
(array([ 958,  965,  966,  967,  981,  983, 1000, 1011, 1070, 1071, 1072,
       1073, 1074, 1075, 1077, 1078, 1079, 1080, 1105, 1124, 1129, 1130,
       1143, 1190, 1252, 1282, 1296, 1299, 1314, 1328, 2118]), array([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4]))
Empty DataFrame
Columns: [(Close, ^NSEI), (High, ^NSEI), (Low, ^NSEI

In [ ]:
print(data.head())

Price             Close         High          Low         Open  Volume
Ticker            ^NSEI        ^NSEI        ^NSEI        ^NSEI   ^NSEI
Date                                                                  
2015-10-21  8251.700195  8294.400391  8217.150391  8258.349609  144800
2015-10-23  8295.450195  8328.099609  8280.750000  8308.250000  152000
2015-10-26  8260.549805  8336.299805  8252.049805  8333.650391  133900
2015-10-27  8232.900391  8241.950195  8217.049805  8230.349609  156700
2015-10-28  8171.200195  8209.099609  8131.799805  8188.899902  188900


In [ ]:
import pandas as pd
import numpy as np
from scipy import signal
from scipy.fft import fft, fftfreq

# -------------------- FLATTENING COLUMNS --------------------
flat_columns = []
for col in data.columns:
    if isinstance(col, tuple):
        name = next((str(c) for c in col if str(c).strip() != ""), "")
    else:
        name = str(col)
    flat_columns.append(name.strip().capitalize())
data.columns = flat_columns

required_cols = {'Open', 'High', 'Low', 'Close', 'Volume'}
missing = required_cols - set(data.columns)
if missing:
    raise ValueError(f"Missing required columns: {missing}")


# -------------------- CROSS-MARKET FEATURES --------------------
def add_market_context_features(data, market_data):
    for key, series in market_data.items():
        aligned_series = series.reindex(data.index, method='ffill')
        data[f'{key}_returns'] = aligned_series.pct_change()
        data[f'{key}_returns_5d'] = aligned_series.pct_change(5)
        data[f'{key}_vs_nifty'] = data['returns'] - data[f'{key}_returns']
        data[f'{key}_correlation_20d'] = data['returns'].rolling(20).corr(data[f'{key}_returns'])
        data[f'{key}_returns_lag1'] = data[f'{key}_returns'].shift(1)
    return data


# -------------------- ENHANCED VOLATILITY FEATURES --------------------
def add_volatility_features(data):
    data['volatility_5d'] = data['returns'].rolling(5).std()
    data['volatility_10d'] = data['returns'].rolling(10).std()
    data['volatility_20d'] = data['returns'].rolling(20).std()
    data['volatility_60d'] = data['returns'].rolling(60).std()

    data['vol_ratio_5_20'] = data['volatility_5d'] / data['volatility_20d']
    data['vol_ratio_10_60'] = data['volatility_10d'] / data['volatility_60d']
    data['atr_pct'] = (data['atr_14'] / data['Close']) * 100
    data['vol_breakout'] = (data['volatility_5d'] > data['volatility_20d'].shift(1) * 1.5).astype(int)
    data['price_vol_divergence'] = (data['returns'].abs() - data['volatility_10d']) / data['volatility_10d']

    data['gk_volatility'] = np.sqrt(
        0.5 * (np.log(data['High'] / data['Low']))**2 -
        (2 * np.log(2) - 1) * (np.log(data['Close'] / data['Open']))**2
    )
    return data


# -------------------- MOMENTUM DIVERGENCE FEATURES --------------------
def add_momentum_divergence_features(data):
    price_momentum_10 = (data['Close'] - data['Close'].shift(10)) / data['Close'].shift(10)
    rsi_momentum_10 = data['rsi_14'] - data['rsi_14'].shift(10)

    data['price_rsi_divergence'] = np.where(
        (price_momentum_10 > 0) & (rsi_momentum_10 < 0), 1,
        np.where((price_momentum_10 < 0) & (rsi_momentum_10 > 0), -1, 0)
    )

    volume_momentum_10 = (data['Volume'] - data['Volume'].shift(10)) / data['Volume'].shift(10)
    data['price_volume_divergence'] = np.where(
        (price_momentum_10 > 0) & (volume_momentum_10 < 0), 1,
        np.where((price_momentum_10 < 0) & (volume_momentum_10 > 0), -1, 0)
    )

    macd_momentum_10 = data['macd'] - data['macd'].shift(10)
    data['price_macd_divergence'] = np.where(
        (price_momentum_10 > 0) & (macd_momentum_10 < 0), 1,
        np.where((price_momentum_10 < 0) & (macd_momentum_10 > 0), -1, 0)
    )

    data['momentum_strength'] = abs(price_momentum_10) / (data['volatility_10d'] + 1e-8)
    return data


# -------------------- PRICE AND TREND FEATURES --------------------
data['returns'] = data['Close'].pct_change()
data = add_market_context_features(data, market_data)
data['log_returns'] = np.log(data['Close'] / data['Close'].shift(1))
data['momentum_3'] = data['Close'] / data['Close'].shift(3) - 1
data['momentum_10'] = data['Close'] / data['Close'].shift(10) - 1
data['roc_5'] = (data['Close'] - data['Close'].shift(5)) / data['Close'].shift(5)
data['roc_10'] = (data['Close'] - data['Close'].shift(10)) / data['Close'].shift(10)
data['price_change_1d'] = data['Close'].diff()
data['price_change_5d'] = data['Close'].diff(5)


# -------------------- MOVING AVERAGES & TREND --------------------
data['sma_5'] = data['Close'].rolling(5).mean()
data['sma_10'] = data['Close'].rolling(10).mean()
data['sma_20'] = data['Close'].rolling(20).mean()
data['sma_50'] = data['Close'].rolling(50).mean()
data['ema_12'] = data['Close'].ewm(span=12, adjust=False).mean()
data['ema_26'] = data['Close'].ewm(span=26, adjust=False).mean()
data['price_above_sma20'] = (data['Close'] > data['sma_20']).astype(int)
data['price_above_sma50'] = (data['Close'] > data['sma_50']).astype(int)


# -------------------- MOMENTUM INDICATORS --------------------
def rsi(series, window=14):
    delta = series.diff()
    gain = delta.clip(lower=0)
    loss = (-delta).clip(lower=0)
    roll_up = gain.rolling(window).mean()
    roll_down = loss.rolling(window).mean()
    rs = roll_up / roll_down
    return 100 - (100 / (1 + rs))

data['rsi_14'] = rsi(data['Close'], 14)
data['rsi_30'] = rsi(data['Close'], 30)
data['rsi_14_lag1'] = data['rsi_14'].shift(1)

ema_12 = data['ema_12']
ema_26 = data['ema_26']
data['macd'] = ema_12 - ema_26
data['macd_signal'] = data['macd'].ewm(span=9, adjust=False).mean()
data['macd_histogram'] = data['macd'] - data['macd_signal']
data['macd_lag1'] = data['macd'].shift(1)


# -------------------- WILLIAMS %R & STOCHASTIC --------------------
hh_14 = data['High'].rolling(14).max()
ll_14 = data['Low'].rolling(14).min()
data['williams_r'] = -100 * (hh_14 - data['Close']) / (hh_14 - ll_14)
data['stoch_k'] = 100 * (data['Close'] - ll_14) / ((hh_14 - ll_14).replace(0, np.nan))


# -------------------- BOLLINGER BANDS & VOLATILITY --------------------
bb_mid = data['Close'].rolling(20).mean()
bb_std = data['Close'].rolling(20).std()
data['bb_middle'] = bb_mid
data['bb_upper'] = bb_mid + 2 * bb_std
data['bb_lower'] = bb_mid - 2 * bb_std
data['bb_width'] = (data['bb_upper'] - data['bb_lower']) / bb_mid
den = (data['bb_upper'] - data['bb_lower']).replace(0, np.nan)
data['bb_position'] = (data['Close'] - data['bb_lower']) / den


# -------------------- ATR (AVERAGE TRUE RANGE) --------------------
tr1 = data['High'] - data['Low']
tr2 = (data['High'] - data['Close'].shift(1)).abs()
tr3 = (data['Low'] - data['Close'].shift(1)).abs()
true_range = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
data['atr_14'] = true_range.rolling(14).mean()
data['atr_14_lag1'] = data['atr_14'].shift(1)
data = add_volatility_features(data)


# -------------------- VOLUME FEATURES --------------------
data['volume_sma_10'] = data['Volume'].rolling(10).mean()
data['volume_ratio'] = data['Volume'] / data['volume_sma_10']
data['obv'] = (np.sign(data['Close'].diff()) * data['Volume']).fillna(0).cumsum()
data['volume_ratio_lag1'] = data['volume_ratio'].shift(1)


# -------------------- TIME-BASED FEATURES --------------------
data['dayofweek'] = data.index.dayofweek if hasattr(data.index, 'dayofweek') else np.nan
data['month'] = data.index.month if hasattr(data.index, 'month') else np.nan


# -------------------- SPECTRAL & CYCLICAL FEATURES --------------------
def add_spectral_features(data, window=60):
    """Add spectral analysis and cyclical pattern features"""

    def rolling_fft_features(series, window):
        dominant_freq, spectral_energy, spectral_centroid = [], [], []

        for i in range(window, len(series)):
            window_data = series.iloc[i - window:i].values
            fft_vals = fft(window_data)
            frequencies = fftfreq(len(window_data))
            magnitude = np.abs(fft_vals[:len(fft_vals)//2])
            freq_positive = frequencies[:len(frequencies)//2]

            if len(magnitude) > 0:
                dominant_idx = np.argmax(magnitude)
                dominant_freq.append(freq_positive[dominant_idx])
                spectral_energy.append(np.sum(magnitude**2))
                if np.sum(magnitude) > 0:
                    spectral_centroid.append(np.sum(freq_positive * magnitude) / np.sum(magnitude))
                else:
                    spectral_centroid.append(0)
            else:
                dominant_freq.append(0)
                spectral_energy.append(0)
                spectral_centroid.append(0)

        return dominant_freq, spectral_energy, spectral_centroid

    dom_freq, spec_energy, spec_centroid = rolling_fft_features(data['returns'].fillna(0), window)
    data['dominant_frequency'] = [np.nan] * window + dom_freq
    data['spectral_energy'] = [np.nan] * window + spec_energy
    data['spectral_centroid'] = [np.nan] * window + spec_centroid

    data['day_of_year'] = data.index.dayofyear if hasattr(data.index, 'dayofyear') else 1
    data['week_of_year'] = data.index.isocalendar().week if hasattr(data.index, 'isocalendar') else 1

    data['yearly_cycle_sin'] = np.sin(2 * np.pi * data['day_of_year'] / 365.25)
    data['yearly_cycle_cos'] = np.cos(2 * np.pi * data['day_of_year'] / 365.25)
    data['monthly_cycle_sin'] = np.sin(2 * np.pi * data.index.day / 30.44)
    data['monthly_cycle_cos'] = np.cos(2 * np.pi * data.index.day / 30.44)
    data['weekly_cycle_sin'] = np.sin(2 * np.pi * data.index.dayofweek / 7) if hasattr(data.index, 'dayofweek') else 0
    data['weekly_cycle_cos'] = np.cos(2 * np.pi * data.index.dayofweek / 7) if hasattr(data.index, 'dayofweek') else 0

    return data


#MACRO FEATURES 
def add_macro_features(data):
    """Add macro-economic context features"""
    data['bull_bear_regime'] = np.where(
        data['sma_50'] > data['sma_50'].shift(20), 1,
        np.where(data['sma_50'] < data['sma_50'].shift(20), -1, 0)
    )

    data['vol_regime'] = np.where(
        data['india_vix_returns'] > data['india_vix_returns'].quantile(0.75), 1,
        np.where(data['india_vix_returns'] < data['india_vix_returns'].quantile(0.25), -1, 0)
    )

    data['global_flows_proxy'] = (data['usdinr_returns'].rolling(5).mean() * -1)
    return data


# -------------------- APPLY FINAL FEATURE BLOCKS --------------------
data = add_macro_features(data)
data = add_spectral_features(data)
data = add_volatility_features(data)
data = add_momentum_divergence_features(data)

# -------------------- FINAL CLEANUP --------------------
data = data.dropna().copy()


/tmp/ipykernel_29461/662640562.py:71: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['price_macd_divergence'] = np.where(
/tmp/ipykernel_29461/662640562.py:76: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['momentum_strength'] = abs(price_momentum_10) / (data['volatility_10d'] + 1e-8)


In [ ]:
data = data.sort_index()

#now we will split the dataset chronologicalyy 80  to 20 %
split_index = int(len(data) * 0.8)

train_data = data.iloc[:split_index].copy()
test_data = data.iloc[split_index:].copy()

# for verification
print(f"Training data range: {train_data.index[0]} to {train_data.index[-1]} ({len(train_data)} rows)")
print(f"Test data range: {test_data.index[0]} to {test_data.index[-1]} ({len(test_data)} rows)")


Training data range: 2016-01-21 00:00:00 to 2023-05-03 00:00:00 (1712 rows)
Test data range: 2023-05-04 00:00:00 to 2025-01-27 00:00:00 (429 rows)


In [ ]:
import numpy as np
import pandas as pd

train_data = train_data.copy()
test_data = test_data.copy()

# Calculate rolling volatility (20-day window recommended)
train_data['volatility_20d'] = train_data['returns'].rolling(20).std()
test_data['volatility_20d'] = test_data['returns'].rolling(20).std()

# Set your risk regime threshold (top 20% volatility = high risk)
vol_threshold = pd.concat([train_data['volatility_20d'], test_data['volatility_20d']]).quantile(0.80)

# Target: 1 = high risk regime (future vol > threshold), 0 = normal
train_data['target'] = (train_data['volatility_20d'].shift(-1) > vol_threshold).astype(int)
test_data['target'] = (test_data['volatility_20d'].shift(-1) > vol_threshold).astype(int)

# Drop last row (now only 1-step lookahead, but you can use more)
train_data = train_data.iloc[:-1]
test_data = test_data.iloc[:-1]


train_data = train_data.dropna(subset=['target', 'volatility_20d'])
test_data = test_data.dropna(subset=['target', 'volatility_20d'])



In [ ]:
#model training in this we are going to ensmble xgboost  , randomforest  , lightbm 
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


#hyperparameter tuning 
xgb_tuning = {'n_estimators': [200, 300, 400],
    'max_depth': [4, 6, 8],
    'learning_rate': [0.01, 0.03, 0.05, 0.1],
    'subsample': [0.7, 0.8, 1.0],
    'colsample_bytree': [0.7, 0.8, 1.0],
    'reg_alpha': [0, 0.1, 0.5],
    'reg_lambda': [1, 1.5, 2] ,
    'scale_pos_weight': [1, 2, 3] 
    }  
rf_tuning ={'n_estimators': [200, 300, 400],
    'max_depth': [6, 8, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
    }
lgbm_tunung = {'n_estimators': [300, 400, 500],
    'learning_rate': [0.01, 0.03, 0.05],
    'max_depth': [-1, 6, 8],
    'num_leaves': [31, 50, 70],
    'subsample': [0.7, 0.8, 1.0],
    'colsample_bytree': [0.7, 0.8, 1.0],
    'reg_alpha': [0, 0.1, 0.5],
    'reg_lambda': [0, 0.1, 0.5]
    }

#xgboost
xgb_model = XGBClassifier(n_estimators=300,
 learning_rate=0.05,
 max_depth=6,
  random_state=42, 
  scale_pos_weight=2 
   )
#random forest
rf_model = RandomForestClassifier(n_estimators=300,
 max_depth=8,
  random_state=42,class_weight= 'balanced',
   n_jobs=-1 , 
     )
#lightgbm
lgbm_model = LGBMClassifier(
    n_estimators=300,          # Slightly fewer trees, faster, reduces overfitting
    learning_rate=0.05,        # Slightly higher for faster convergence
    max_depth=-1,              # No max depth limit
    num_leaves=31,             # Optimal for small datasets, reduces overfitting
    min_child_samples=20,      # Increase to avoid overfitting small leaf nodes
    subsample=0.9,             # Row sampling for bagging
    colsample_bytree=0.8,      # Feature sampling
    reg_alpha=0.2,             # L1 regularization
    reg_lambda=0.2,            # L2 regularization
    class_weight='balanced',   
    force_col_wise=True,
    random_state=42,
    n_jobs=-1                  
)

#using randomizedSearchcv 
from sklearn.model_selection import RandomizedSearchCV  
xgb_randomized = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions= xgb_tuning, 
    n_iter=10,
    cv=3,
    scoring='accuracy',
    n_jobs=-1,
    random_state=42
)

rf_randomized = RandomizedSearchCV(
 estimator=rf_model,
    param_distributions=rf_tuning,
    n_iter=10,
    cv=3,
    scoring='accuracy',
    n_jobs=-1,
    random_state=42
)

lgbm_randomized = RandomizedSearchCV(
     estimator=lgbm_model,
    param_distributions=lgbm_tunung,
    n_iter=10,
    cv=3,
    scoring='accuracy',
    n_jobs=-1,
    random_state=42
)


# Features to use for training (exclude target and date)
feature_cols = [
    'returns', 'log_returns', 'momentum_3', 'momentum_10', 'roc_5', 'roc_10',
    'price_change_1d', 'price_change_5d',
    'sma_5', 'sma_10', 'sma_20', 'sma_50', 'ema_12', 'ema_26',
    'price_above_sma20', 'price_above_sma50',
    'rsi_14', 'rsi_30', 'rsi_14_lag1',
    'macd', 'macd_signal', 'macd_histogram', 'macd_lag1',
    'williams_r', 'stoch_k',
    'bb_middle', 'bb_upper', 'bb_lower', 'bb_width', 'bb_position',
    'atr_14', 'atr_14_lag1',
    'volume_sma_10', 'volume_ratio', 'obv', 'volume_ratio_lag1',
    'bank_nifty_returns', 'bank_nifty_vs_nifty', 'bank_nifty_correlation_20d',
    'india_vix_returns', 'india_vix_vs_nifty', 'india_vix_correlation_20d',
    'sp500_returns', 'sp500_vs_nifty', 'usdinr_returns', 'crude_oil_returns',
    'gold_returns',
    'volatility_5d', 'volatility_10d', 'volatility_20d', 'volatility_60d',
    'vol_ratio_5_20', 'vol_ratio_10_60', 'atr_pct', 'vol_breakout',
    'price_vol_divergence', 'gk_volatility','price_rsi_divergence', 
    'price_volume_divergence', 'price_macd_divergence',
    'momentum_strength','dominant_frequency', 'spectral_energy', 'spectral_centroid',
    'yearly_cycle_sin', 'yearly_cycle_cos', 'monthly_cycle_sin', 'monthly_cycle_cos',
    'weekly_cycle_sin', 'weekly_cycle_cos'
]

X_train = train_data[feature_cols].copy()
y_train = train_data['target'].copy()
X_test = test_data[feature_cols].copy()
y_test = test_data['target'].copy()
 
 #Using smote  + tmoek
from imblearn.combine import SMOTETomek
sm = SMOTETomek(random_state=42)
X_train_bal, y_train_bal = sm.fit_resample(X_train, y_train)


xgb_randomized.fit(X_train, y_train)
rf_randomized.fit(X_train, y_train)
lgbm_randomized.fit(X_train, y_train)

#combineing all three into metaaa model 
from sklearn.ensemble import StackingClassifier  
from sklearn.linear_model import LogisticRegression

#defineing estimators for the meta model 
estimators =[('xgb', xgb_randomized.best_estimator_) , ('rf',rf_randomized.best_estimator_) , ('lgbm',lgbm_randomized.best_estimator_)]  

#making own meta hybrid model
StackHybrid_model = StackingClassifier(
estimators = estimators ,
final_estimator =LogisticRegression(max_iter=1000, random_state=42),
cv= 5 , 
passthrough= True ,
n_jobs = -1)   

StackHybrid_model.fit(X_train, y_train)

#predictions and evaluations
y_pred = StackHybrid_model .predict(X_test)
y_pred_proba = StackHybrid_model.predict_proba(X_test)
acc = accuracy_score(y_test, y_pred)

#Model detailed report 
print(f"\nENHANCED MODEL PERFORMANCE:")
print(f"Test Accuracy: {acc:.4f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("\n Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

#  FEATURE IMPORTANCE FROM BEST MODELS
print("\nTop 10 Most Important Features:")
if hasattr(xgb_randomized.best_estimator_, 'feature_importances_'):
    feature_importance = pd.DataFrame({
        'feature': feature_cols,
        'importance': xgb_randomized.best_estimator_.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print(feature_importance.head(10))

#  INDIVIDUAL MODEL PERFORMANCE
print("\n Individual Model Performance:")
for name, model in estimators:
    individual_pred = model.predict(X_test)
    individual_acc = accuracy_score(y_test, individual_pred)
   
    print(f"{name.upper()}: Accuracy={individual_acc:.4f}")

[LightGBM] [Info] Number of positive: 266, number of negative: 862
[LightGBM] [Info] Total Bins 15143
[LightGBM] [Info] Number of data points in the train set: 1128, number of used features: 70
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Number of positive: 267, number of negative: 861
[LightGBM] [Info] Start training from score 0.000000
[LightGBM] [Info] Total Bins 15148
[LightGBM] [Info] Number of positive: 267, number of negative: 861
[LightGBM] [Info] Number of positive: 267, number of negative: 861
[LightGBM] [Info] Number of data points in the train set: 1128, number of used features: 70
[LightGBM] [Info] Total Bins 15140
[LightGBM] [Info] Total Bins 15140
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
[LightGBM] [Info] Number of data points in the train set: 1128, numbe

/home/sanket/venv/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



ENHANCED MODEL PERFORMANCE:
Test Accuracy: 0.9829

Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       389
           1       1.00      0.65      0.79        20

    accuracy                           0.98       409
   macro avg       0.99      0.82      0.89       409
weighted avg       0.98      0.98      0.98       409


 Confusion Matrix:
[[389   0]
 [  7  13]]

Top 10 Most Important Features:
           feature  importance
49  volatility_20d    0.394563
4            roc_5    0.025335
12          ema_12    0.023858
21  macd_histogram    0.022778
3      momentum_10    0.022236
53         atr_pct    0.022057
18     rsi_14_lag1    0.019248
9           sma_10    0.017904
31     atr_14_lag1    0.017736
29     bb_position    0.016226

 Individual Model Performance:
XGB: Accuracy=0.9951
RF: Accuracy=0.9853
LGBM: Accuracy=0.9951
